# rapid_textrank Quick Start

This notebook demonstrates the basics of `rapid_textrank`, a high-performance TextRank implementation in Rust with Python bindings.

**What you'll learn:**
- How to extract keywords from text
- Understanding `Phrase` objects and their attributes
- Configuring the algorithm with `TextRankConfig`
- Multi-language support
- Optional spaCy integration

## Installation

Install `rapid_textrank` from PyPI:

In [ ]:
%pip install -q rapid_textrank

## Basic Usage

The simplest way to extract keywords is with the `extract_keywords()` function:

In [ ]:
from rapid_textrank import extract_keywords

text = """
Machine learning is a subset of artificial intelligence that enables
systems to learn and improve from experience. Deep learning, a type of
machine learning, uses neural networks with many layers.
"""

# Extract the top 5 keywords
keywords = extract_keywords(text, top_n=5, language="en")

for phrase in keywords:
    print(f"{phrase.text}: {phrase.score:.4f}")

## Understanding Results: The `Phrase` Object

Each keyword is returned as a `Phrase` object with several useful attributes:

In [ ]:
# Let's examine the first phrase in detail
phrase = keywords[0]

print("Phrase attributes:")
print(f"  text:  {phrase.text!r:30} # The original phrase text")
print(f"  lemma: {phrase.lemma!r:30} # Lemmatized (base) form")
print(f"  score: {phrase.score:<30.4f} # TextRank importance score")
print(f"  rank:  {phrase.rank:<30} # 1-indexed ranking position")
print(f"  count: {phrase.count:<30} # Number of occurrences in text")

In [ ]:
# View all keywords as a table
print(f"{'Rank':<6} {'Text':<35} {'Score':<10} {'Count':<6}")
print("-" * 60)
for p in keywords:
    print(f"{p.rank:<6} {p.text:<35} {p.score:<10.4f} {p.count:<6}")

## Configuration with `TextRankConfig`

For more control over the algorithm, use `TextRankConfig` with the class-based API:

In [ ]:
from rapid_textrank import TextRankConfig, BaseTextRank

# Create a custom configuration
config = TextRankConfig(
    damping=0.85,              # PageRank damping factor (0-1)
    max_iterations=100,        # Maximum PageRank iterations
    convergence_threshold=1e-6,# Stop when scores change less than this
    window_size=4,             # Co-occurrence window size
    top_n=10,                  # Number of results
    min_phrase_length=1,       # Minimum words in a phrase
    max_phrase_length=4,       # Maximum words in a phrase
    score_aggregation="sum",   # How to combine word scores: "sum", "mean", "max", "rms"
    language="en"              # Language for stopwords
)

# Create an extractor with the config
extractor = BaseTextRank(config=config)

# Extract keywords
result = extractor.extract_keywords(text)

print(f"Converged: {result.converged}")
print(f"Iterations: {result.iterations}")
print(f"\nTop phrases:")
for p in result.phrases[:5]:
    print(f"  {p.rank}. {p.text}: {p.score:.4f}")

### Key Configuration Parameters

| Parameter | Default | Description |
|-----------|---------|-------------|
| `damping` | 0.85 | PageRank damping factor. Higher = more weight to graph structure |
| `window_size` | 4 | Co-occurrence window. Larger = more connections between distant words |
| `min_phrase_length` | 1 | Allow single words as phrases |
| `max_phrase_length` | 4 | Maximum words in a keyphrase |
| `score_aggregation` | "sum" | How to combine word scores: sum, mean, max, or rms |

In [ ]:
# Compare different score aggregation methods
aggregation_methods = ["sum", "mean", "max", "rms"]

print("Score Aggregation Comparison (top 3 phrases):")
print("=" * 60)

for method in aggregation_methods:
    config = TextRankConfig(score_aggregation=method, top_n=3, language="en")
    extractor = BaseTextRank(config=config)
    result = extractor.extract_keywords(text)
    
    print(f"\n{method.upper()}:")
    for p in result.phrases:
        print(f"  {p.text}: {p.score:.4f}")

## Multi-Language Support

`rapid_textrank` supports stopword filtering for 18 languages:

| Code | Language | Code | Language | Code | Language |
|------|----------|------|----------|------|----------|
| `en` | English | `de` | German | `fr` | French |
| `es` | Spanish | `it` | Italian | `pt` | Portuguese |
| `nl` | Dutch | `ru` | Russian | `sv` | Swedish |
| `no` | Norwegian | `da` | Danish | `fi` | Finnish |
| `hu` | Hungarian | `tr` | Turkish | `pl` | Polish |
| `ar` | Arabic | `zh` | Chinese | `ja` | Japanese |

In [ ]:
# German example
german_text = """
Maschinelles Lernen ist ein Teilgebiet der künstlichen Intelligenz.
Deep Learning verwendet neuronale Netze mit vielen Schichten.
Diese Technologie ermöglicht es Computern, aus Erfahrung zu lernen.
"""

keywords_de = extract_keywords(german_text, top_n=5, language="de")

print("German keywords:")
for p in keywords_de:
    print(f"  {p.rank}. {p.text}: {p.score:.4f}")

In [ ]:
# French example
french_text = """
L'apprentissage automatique est une branche de l'intelligence artificielle.
Les réseaux de neurones profonds permettent l'analyse de données complexes.
Ces technologies transforment de nombreux secteurs industriels.
"""

keywords_fr = extract_keywords(french_text, top_n=5, language="fr")

print("French keywords:")
for p in keywords_fr:
    print(f"  {p.rank}. {p.text}: {p.score:.4f}")

## Optional: spaCy Integration

If you use spaCy, `rapid_textrank` can integrate as a pipeline component. This lets you use spaCy's superior tokenization with rapid_textrank's fast extraction.

**Note:** spaCy requires Python 3.9-3.12. If you're using Python 3.13+, spaCy may not be available yet.

First, install the required dependencies:

In [ ]:
# Uncomment to install spaCy dependencies
# %pip install -q spacy
# !python3 -m spacy download en_core_web_sm

In [ ]:
# spaCy integration example
try:
    import spacy
    import rapid_textrank.spacy_component  # Registers the pipeline factory
    
    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("rapid_textrank")
    
    doc = nlp(text)
    
    print("Keywords via spaCy pipeline:")
    for phrase in doc._.phrases[:5]:
        print(f"  {phrase.rank}. {phrase.text}: {phrase.score:.4f}")
        
except ImportError:
    print("spaCy not installed. This is optional - rapid_textrank works great on its own!")
except OSError:
    print("spaCy model not found. Run: python3 -m spacy download en_core_web_sm")

## Next Steps

Now that you know the basics, explore more in the other notebooks:

1. **[02_algorithm_variants.ipynb](02_algorithm_variants.ipynb)** - Deep dive into BaseTextRank, PositionRank, and BiasedTextRank
2. **[03_explain_algorithm.ipynb](03_explain_algorithm.ipynb)** - Visual explanation of how TextRank works
3. **[04_benchmarks.ipynb](04_benchmarks.ipynb)** - Performance comparison with pytextrank

### Resources

- [GitHub Repository](https://github.com/xang1234/rapid-textrank)
- [Original TextRank Paper](https://aclanthology.org/W04-3252/) (Mihalcea & Tarau, 2004)